In [ ]:
!pip install pygis
!pip install cartopy

Reference to this module: [cartoee](https://geemap.org/notebooks/50_cartoee_quickstart/)

In [ ]:
%pylab inline
import ee 
import geemap
from geemap import cartoee

In [ ]:
geemap.ee_initialize()

# Plot Image: Singleband

In [ ]:
srtm = ee.Image("CGIAR/SRTM90_V4")
region = [180, -60, -180, 85]
vis = {'min': 0, 'max': 3000}

In [ ]:
fig = plt.figure(figsize=(15, 9))

ax = cartoee.get_map(srtm, region=region, vis_params=vis)
cartoee.add_colorbar(ax, vis, loc="bottom", label="Elevation (m)", orientation="horizontal")
cartoee.add_gridlines(ax, interval=[60, 30], linestyle=":")
ax.coastlines(color="red")

plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 7))

cmap = "terrain"

ax = cartoee.get_map(srtm, region=region, vis_params=vis, cmap=cmap)
cartoee.add_colorbar(
    ax, vis, cmap=cmap, loc="right", label="Elevation (m)", orientation="vertical"
)

cartoee.add_gridlines(ax, interval=[60, 30], linestyle="--")
ax.coastlines(color="red")
ax.set_title(label='Global Elevation Map', fontsize=15)

plt.show()

In [ ]:
cartoee.savefig(fig, fname="srtm.jpg", dpi=300, bbox_inches='tight')

# Plot Image: Multi-spectral

In [ ]:
ROI = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(ee.Filter.eq('ADM0_NAME', 'Cambodia'))

def mask_clouds(image):
  qa = image.select('QA60')

  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  mask = (
      qa.bitwiseAnd(cloud_bit_mask).eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

def clip(image):
    return image.clip(ROI)

img_collect = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(ROI)
    .filterDate('2023-01-01', '2023-03-28')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_clouds)
    .map(clip)
)

image = img_collect.reduce(ee.Reducer.percentile([20])).select(['B4_p20', 'B3_p20', 'B2_p20'])
image

In [ ]:
fig = plt.figure(figsize=(15, 10))

vis = {"bands": ['B4_p20', 'B3_p20', 'B2_p20'], "min": 0.025, "max": 0.25, "gamma": 1.3}

bbox = [102.32, 10.29, 108.06, 14.96]

ax = cartoee.get_map(image, region=bbox, vis_params=vis)

cartoee.pad_view(ax)
cartoee.add_gridlines(ax, interval=0.5, xtick_rotation=0, linestyle=":")

plt.show()

In [ ]:
cartoee.savefig(fig, fname="cambodia-image.jpg", dpi=300, bbox_inches='tight')

# Adding more features of cartography

In [ ]:
fig = plt.figure(figsize=(15, 10))

region = [102.32, 10.29, 108.06, 14.96]
ax = cartoee.get_map(image, vis_params=vis, region=region)
cartoee.add_gridlines(ax, interval=0.5, xtick_rotation=0, linestyle=":")
cartoee.add_north_arrow(
    ax, text="N", xy=(0.05, 0.25), text_color="white", arrow_color="white", fontsize=20
)
cartoee.add_scale_bar_lite(
    ax, length=10, xy=(0.1, 0.05), fontsize=20, color="white", unit="km"
)
ax.set_title(label='Cambodia Satellite Images (period: January - March 2023)', fontsize=15)

plt.show()

cartoee.savefig(fig, fname="cambodia-image.jpg", dpi=300, bbox_inches='tight')

# Seasonal NDVI

In [ ]:
import ee
import cartoee as cee